In [40]:
import sys
sys.path.append("..")

from ResidualAttentionNetwork import ResidualAttentionNetwork

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import random
import os
from PIL import Image
from tqdm import tqdm
import h5py

import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Conv2DTranspose, Flatten, Reshape, Attention
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers,Sequential
from keras.models import Model
import keras
import time

import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 


False

In [41]:
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

batch_size=8

epochs = 100

In [42]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=.3
)

In [43]:
train_data_dir="./nerthus-dataset-frames/"
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data




Found 3868 images belonging to 4 classes.
Found 1657 images belonging to 4 classes.


In [137]:

latent_size=64

loss_fn1 = keras.losses.MeanSquaredError()
loss_fn2 = keras.losses.CategoricalCrossentropy()
optimizer=optimizers.Adam(lr=0.01)
# Prepare the metrics.
train_acc_metric1 = keras.metrics.MeanSquaredError()
val_acc_metric1 = keras.metrics.MeanSquaredError()
train_acc_metric2 = keras.metrics.CategoricalAccuracy()
val_acc_metric2 = keras.metrics.CategoricalAccuracy()

input_img = Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))  
# You can experiment with the encoder layers, i.e. add or change them
x = Conv2D(16, (3, 3), activation='relu', strides=2, padding='same')(input_img)
x = Conv2D(32, (3, 3), activation='relu', strides=2, padding='same')(x)    
encoded_shape = x.shape
x = Flatten()(x)
encoded = Dense(latent_size)(x)
encoderModel = Model(input_img,encoded,name='encoder')

encoded_input = Input(shape=(latent_size,))
x = Dense(np.prod(encoded_shape[1:]))(encoded_input)
x = Reshape((encoded_shape[1], encoded_shape[2], encoded_shape[3]))(x)
x = Conv2DTranspose(32,(3, 3), activation='relu',strides=2, padding='same')(x)
x = Conv2DTranspose(16,(3, 3), activation='relu', strides=2, padding='same')(x)
outputDecoder = Conv2DTranspose(3,(3, 3), activation='sigmoid', padding='same')(x)
decoderModel = Model(encoded_input,outputDecoder,name='decoder')
autoencoder = Model(input_img, decoderModel(encoderModel(input_img)),name="autoencoder")
# autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy')

inputLayer = Input(shape=(latent_size,))
encoded_input1=tf.expand_dims(inputLayer,2)
encoded_input1=tf.matmul(encoded_input1, tf.transpose(encoded_input1,perm=[0,2,1]))
encoded=tf.expand_dims(encoded_input1,3)
# encoded_input1.shape
output=ResidualAttentionNetwork((latent_size, encoded.shape[2], 1), 4, activation='softmax').build_model_for_latent_input(encoded)
classificationModel = Model(inputs=inputLayer, outputs=output,name="classification")

# classificationModel.compile(optimizer=optimizer,
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])


# Train

In [49]:

nb_epochs=10

@tf.function
def train_step(x):
    with tf.GradientTape() as tape:
        logits = autoencoder(x, training=True)
        loss_value = loss_fn1(x, logits)
    grads = tape.gradient(loss_value, autoencoder.trainable_weights)
    optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))
    train_acc_metric1.update_state(x, logits)
    return loss_value

@tf.function
def test_step(x):
    val_logits = autoencoder(x, training=False)
    val_acc_metric1.update_state(x, val_logits)

for epoch in range(nb_epochs):
    print("\nStart of epoch %d" % (epoch,))
    for step in tqdm(range(200)):

        (x_batch_train, y_batch_train)=next(train_generator)
        loss_value = train_step(x_batch_train)

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric1.result()
    print("Training mse over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric1.reset_states()

    # Run a validation loop at the end of each epoch.
    for indexVal in range(20):
        x_batch_val, y_batch_val=next(validation_generator)
        test_step(x_batch_val)

    val_acc = val_acc_metric1.result()
    val_acc_metric1.reset_states()
    print("Validation mse: %.4f" % (float(val_acc),))
   
        
    

  0%|          | 0/200 [00:00<?, ?it/s]
Start of epoch 0
  1%|          | 2/200 [00:00<01:58,  1.68it/s]Training loss (for one batch) at step 0: 0.0967
Seen so far: 64 samples
 26%|██▌       | 52/200 [00:06<00:16,  9.09it/s]Training loss (for one batch) at step 50: 0.0594
Seen so far: 3264 samples
 51%|█████     | 102/200 [00:12<00:11,  8.85it/s]Training loss (for one batch) at step 100: 0.0377
Seen so far: 6464 samples
 76%|███████▌  | 152/200 [00:17<00:05,  8.87it/s]Training loss (for one batch) at step 150: 0.0457
Seen so far: 9664 samples
100%|██████████| 200/200 [00:23<00:00,  8.59it/s]
Training acc over epoch: 0.0575
  0%|          | 1/200 [00:00<00:21,  9.37it/s]Validation acc: 0.0433

Start of epoch 1
Training loss (for one batch) at step 0: 0.0409
Seen so far: 64 samples
 26%|██▌       | 52/200 [00:05<00:16,  9.00it/s]Training loss (for one batch) at step 50: 0.0459
Seen so far: 3264 samples
 51%|█████     | 102/200 [00:11<00:10,  8.96it/s]Training loss (for one batch) at step

In [138]:

nb_epochs=10

@tf.function
def train_step(x,y):
    with tf.GradientTape() as tape:
        logits = classificationModel(x, training=True)
        loss_value = loss_fn2(y, logits)
    grads = tape.gradient(loss_value, classificationModel.trainable_weights)
    optimizer.apply_gradients(zip(grads, classificationModel.trainable_weights))
    train_acc_metric2.update_state(y, logits)
    return loss_value

@tf.function
def test_step(x,y):
    val_logits = classificationModel(x, training=False)
    val_acc_metric2.update_state(y, val_logits)

for epoch in range(nb_epochs):
    print("\nStart of epoch %d" % (epoch,))
    for step in tqdm(range(200)):

        (x_batch_train, y_batch_train)=next(train_generator)
        x_batch_train=encoderModel(x_batch_train)
        loss_value = train_step(x_batch_train,y_batch_train)

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric2.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric2.reset_states()

    # Run a validation loop at the end of each epoch.
    for indexVal in range(20):
        x_batch_val, y_batch_val=next(validation_generator)
        x_batch_val=encoderModel(x_batch_val)
        test_step(x_batch_val,y_batch_val)

    val_acc = val_acc_metric2.result()
    val_acc_metric2.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
   
        
    

  0%|          | 0/200 [00:00<?, ?it/s]
Start of epoch 0
100%|██████████| 200/200 [02:31<00:00,  1.32it/s]
Training acc over epoch: 0.4148
  0%|          | 0/200 [00:00<?, ?it/s]Validation acc: 0.4750

Start of epoch 1
100%|██████████| 200/200 [02:22<00:00,  1.40it/s]
Training acc over epoch: 0.4931
  0%|          | 0/200 [00:00<?, ?it/s]Validation acc: 0.4510

Start of epoch 2
100%|██████████| 200/200 [02:22<00:00,  1.41it/s]
Training acc over epoch: 0.5019
  0%|          | 0/200 [00:00<?, ?it/s]Validation acc: 0.4688

Start of epoch 3
100%|██████████| 200/200 [02:20<00:00,  1.42it/s]
Training acc over epoch: 0.4869
  0%|          | 0/200 [00:00<?, ?it/s]Validation acc: 0.1813

Start of epoch 4
100%|██████████| 200/200 [02:21<00:00,  1.42it/s]
Training acc over epoch: 0.4762
  0%|          | 0/200 [00:00<?, ?it/s]Validation acc: 0.5250

Start of epoch 5
100%|██████████| 200/200 [02:22<00:00,  1.40it/s]
Training acc over epoch: 0.4975
  0%|          | 0/200 [00:00<?, ?it/s]Validation a